In [33]:
import click
import torch
import logging
import random
import numpy as np

from utils.config import Config
from utils.visualization.plot_images_grid import plot_images_grid
from deepSVDD import DeepSVDD
from datasets.main import load_dataset


## Preparing config & datasets

### Default Setting

In [34]:
load_config = None
nu = 0.01
seed = -1
n_jobs_dataloader = 0
load_model = None
optimizer_name = 'adam'
ae_optimizer_name = 'adam'
device = 'cuda'
apply_model = True


### CIFAR-10

In [35]:
# dataset_name = 'cifar10'
# net_name = 'cifar10_LeNet'
# xp_path = '../log/cifar10_test'
# data_path = '../data'
# objective = 'one-class'
# lr = 0.0001
# n_epochs = 150
# lr_milestone = [50]
# batch_size = 200
# weight_decay = 0.5e-6
# pretrain = True
# ae_lr = 0.0001
# ae_n_epochs = 350
# ae_lr_milestone = [250]
# ae_batch_size = 200
# ae_weight_decay = 0.5e-6
# normal_class = 3

### MNIST

In [45]:
dataset_name = 'mnist'
net_name = 'mnist_LeNet'
xp_path = '../log/mnist_test'
data_path = '../data'
objective = 'one-class'
lr = 0.0001
n_epochs = 150
lr_milestone = [50]
batch_size = 200
weight_decay = 0.5e-6
pretrain = True
ae_lr = 0.0001
ae_n_epochs = 150
ae_lr_milestone = [50]
ae_batch_size = 200
ae_weight_decay = 0.5e-3
normal_class = 3

## Load Config

In [46]:

# Get configuration
cfg = Config(locals().copy())

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
log_file = xp_path + '/log.txt'
file_handler = logging.FileHandler(log_file)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)


## Add Logs

In [47]:

# Print arguments
logger.info('Log file is %s.' % log_file)
logger.info('Data path is %s.' % data_path)
logger.info('Export path is %s.' % xp_path)

logger.info('Dataset: %s' % dataset_name)
logger.info('Normal class: %d' % normal_class)
logger.info('Network: %s' % net_name)

# If specified, load experiment config from JSON-file
if load_config:
    cfg.load_config(import_json=load_config)
    logger.info('Loaded configuration from %s.' % load_config)

# Print configuration
logger.info('Deep SVDD objective: %s' % cfg.settings['objective'])
logger.info('Nu-paramerter: %.2f' % cfg.settings['nu'])

# Set seed
if cfg.settings['seed'] != -1:
    random.seed(cfg.settings['seed'])
    np.random.seed(cfg.settings['seed'])
    torch.manual_seed(cfg.settings['seed'])
    logger.info('Set seed to %d.' % cfg.settings['seed'])

# Default device to 'cpu' if cuda is not available
if not torch.cuda.is_available():
    device = 'cpu'
logger.info('Computation device: %s' % device)
logger.info('Number of dataloader workers: %d' % n_jobs_dataloader)

INFO:root:Log file is ../log/mnist_test/log.txt.
INFO:root:Data path is ../data.
INFO:root:Export path is ../log/mnist_test.
INFO:root:Dataset: mnist
INFO:root:Normal class: 3
INFO:root:Network: mnist_LeNet
INFO:root:Deep SVDD objective: one-class
INFO:root:Nu-paramerter: 0.01
INFO:root:Computation device: cpu
INFO:root:Number of dataloader workers: 0


## Load Datasets

In [48]:
# Load data
dataset = load_dataset(dataset_name, data_path, normal_class)

## Setup model and network

In [49]:
# Initialize DeepSVDD model and set neural network \phi
deep_SVDD = DeepSVDD(cfg.settings['objective'], cfg.settings['nu'])
deep_SVDD.set_network(net_name)

In [50]:
# If specified, load Deep SVDD model (radius R, center c, network weights, and possibly autoencoder weights)
if load_model:
    deep_SVDD.load_model(model_path=load_model, load_ae=True)
    logger.info('Loading model from %s.' % load_model)

logger.info('Pretraining: %s' % pretrain)

INFO:root:Pretraining: True


### Using Pre-Trained

In [51]:

logger.info('Pretraining: %s' % pretrain)
if pretrain:
    # Log pretraining details
    logger.info('Pretraining optimizer: %s' % cfg.settings['ae_optimizer_name'])
    logger.info('Pretraining learning rate: %g' % cfg.settings['ae_lr'])
    logger.info('Pretraining epochs: %d' % cfg.settings['ae_n_epochs'])
    logger.info('Pretraining learning rate scheduler milestones: %s' % (cfg.settings['ae_lr_milestone'],))
    logger.info('Pretraining batch size: %d' % cfg.settings['ae_batch_size'])
    logger.info('Pretraining weight decay: %g' % cfg.settings['ae_weight_decay'])

    # Pretrain model on dataset (via autoencoder)
    deep_SVDD.pretrain(dataset,
                        optimizer_name=cfg.settings['ae_optimizer_name'],
                        lr=cfg.settings['ae_lr'],
                        n_epochs=cfg.settings['ae_n_epochs'],
                        lr_milestones=cfg.settings['ae_lr_milestone'],
                        batch_size=cfg.settings['ae_batch_size'],
                        weight_decay=cfg.settings['ae_weight_decay'],
                        device=device,
                        n_jobs_dataloader=n_jobs_dataloader)

INFO:root:Pretraining: True
INFO:root:Pretraining optimizer: adam
INFO:root:Pretraining learning rate: 0.0001
INFO:root:Pretraining epochs: 150
INFO:root:Pretraining learning rate scheduler milestones: [50]
INFO:root:Pretraining batch size: 200
INFO:root:Pretraining weight decay: 0.0005
INFO:root:Starting pretraining...
INFO:root:  Epoch 1/150	 Time: 8.529	 Loss: 153.09089734
INFO:root:  Epoch 2/150	 Time: 7.836	 Loss: 104.41770566
INFO:root:  Epoch 3/150	 Time: 7.558	 Loss: 75.85939297
INFO:root:  Epoch 4/150	 Time: 7.579	 Loss: 57.18945239
INFO:root:  Epoch 5/150	 Time: 7.907	 Loss: 44.01152518
INFO:root:  Epoch 6/150	 Time: 7.028	 Loss: 34.53868841
INFO:root:  Epoch 7/150	 Time: 6.223	 Loss: 27.81984766
INFO:root:  Epoch 8/150	 Time: 6.637	 Loss: 23.02433888
INFO:root:  Epoch 9/150	 Time: 6.410	 Loss: 19.59991713
INFO:root:  Epoch 10/150	 Time: 6.422	 Loss: 17.05780146
INFO:root:  Epoch 11/150	 Time: 6.330	 Loss: 15.09773337
INFO:root:  Epoch 12/150	 Time: 6.539	 Loss: 13.54791915
I

KeyboardInterrupt: 

In [ ]:

# Log training details
logger.info('Training optimizer: %s' % cfg.settings['optimizer_name'])
logger.info('Training learning rate: %g' % cfg.settings['lr'])
logger.info('Training epochs: %d' % cfg.settings['n_epochs'])
logger.info('Training learning rate scheduler milestones: %s' % (cfg.settings['lr_milestone'],))
logger.info('Training batch size: %d' % cfg.settings['batch_size'])
logger.info('Training weight decay: %g' % cfg.settings['weight_decay'])


## Train model

In [ ]:
# Train model on dataset
deep_SVDD.train(dataset,
                optimizer_name=cfg.settings['optimizer_name'],
                lr=cfg.settings['lr'],
                n_epochs=cfg.settings['n_epochs'],
                lr_milestones=cfg.settings['lr_milestone'],
                batch_size=cfg.settings['batch_size'],
                weight_decay=cfg.settings['weight_decay'],
                device=device,
                n_jobs_dataloader=n_jobs_dataloader)

## Evaluate

In [ ]:
deep_SVDD.test(dataset, device=device, n_jobs_dataloader=n_jobs_dataloader)

In [ ]:
# Plot most anomalous and most normal (within-class) test samples
indices, labels, scores = zip(*deep_SVDD.results['test_scores'])
indices, labels, scores = np.array(indices), np.array(labels), np.array(scores)
idx_sorted = indices[labels == 0][np.argsort(scores[labels == 0])]  # sorted from lowest to highest anomaly score


In [ ]:
if dataset_name in ('mnist', 'cifar10', 'custom'):

    if dataset_name == 'mnist':
        X_normals = dataset.test_set.test_data[idx_sorted[:32], ...].unsqueeze(1)
        X_outliers = dataset.test_set.test_data[idx_sorted[-32:], ...].unsqueeze(1)

    if dataset_name == 'cifar10':
        X_normals = torch.tensor(np.transpose(dataset.test_set.test_data[idx_sorted[:32], ...], (0, 3, 1, 2)))
        X_outliers = torch.tensor(np.transpose(dataset.test_set.test_data[idx_sorted[-32:], ...], (0, 3, 1, 2)))

    if dataset_name == 'custom':
        X_normals = torch.tensor(dataset.test_set[idx_sorted[:32], ...])
        X_outliers = torch.tensor(dataset.test_set[idx_sorted[-32:], ...])

    plot_images_grid(X_normals, export_img=xp_path + '/normals', title='Most normal examples', padding=2)
    plot_images_grid(X_outliers, export_img=xp_path + '/outliers', title='Most anomalous examples', padding=2)


## Save model

In [ ]:
# Save results, model, and configuration
deep_SVDD.save_results(export_json=xp_path + '/results.json')
deep_SVDD.save_model(export_model=xp_path + '/model.tar')
cfg.save_config(export_json=xp_path + '/config.json')

In [ ]:
# Apply model and print to file
if apply_model:
    deep_SVDD.apply_model(export_file=xp_path + '/apply_output.txt', dataset=dataset, device=device, n_jobs_dataloader=n_jobs_dataloader)
